In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import keras
from keras import layers
import numpy as np
import gym

In [2]:
from vae_recurrent import VAE, create_decoder, create_encoder
from transition_gru import TransitionGRU
from recurrent_agent import DAIFAgentRecurrent

In [3]:
from util import random_observation_sequence, transform_observations

In [4]:
enc = create_encoder(2, 2, [20])
dec = create_decoder(2, 2, [20])
# tran = TransitionGRU(2, 1)

env = gym.make('MountainCarContinuous-v0')

Metal device set to: Apple M1 Pro


2022-08-04 19:50:08.721629: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-04 19:50:08.721756: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [5]:
num_seqs = 1200
seq_length = 12
ob_seqs = []
next_obs = []

for i in range(num_seqs):
    o, a, r = random_observation_sequence(env, seq_length)

    train = np.concatenate([o[:-1], a], axis=1)
    test = o[-1]

    ob_seqs.append(train)
    next_obs.append(test)

ob_seqs = np.array(ob_seqs)
next_obs = np.array(next_obs)
ob_seqs.shape

ob_seqs_stddev = np.ones_like(ob_seqs)
next_obs_stddev = np.ones_like(next_obs)

next_obs

array([[-0.47832373, -0.00274939],
       [-0.49251005, -0.005223  ],
       [-0.57791984, -0.00141938],
       ...,
       [-0.45883128, -0.00531333],
       [-0.48911935, -0.00945645],
       [-0.49597296, -0.00507886]])

In [6]:
m = TransitionGRU(2, 1, 10, 30, 2)

m.compile(optimizer="Adam")
# m.build((None, None, 3))
# m.summary()
init_state = np.zeros((1200, 30))

In [7]:
m.fit((ob_seqs, init_state), (next_obs, next_obs_stddev), batch_size=24, epochs=20)

Epoch 1/20
Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


2022-08-04 19:50:09.305537: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-08-04 19:50:09.625944: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-04 19:50:09.722267: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-08-04 19:50:09.796229: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


50/50 [==============================] - 1s 7ms/step - kl_loss: 0.0653
Epoch 2/20
50/50 [==============================] - 0s 7ms/step - kl_loss: 0.0022
Epoch 3/20
50/50 [==============================] - 0s 7ms/step - kl_loss: 1.5263e-04
Epoch 4/20
50/50 [==============================] - 0s 7ms/step - kl_loss: 1.0990e-04
Epoch 5/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 9.6734e-05
Epoch 6/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 8.1018e-05
Epoch 7/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 7.0442e-05
Epoch 8/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 6.0853e-05
Epoch 9/20
50/50 [==============================] - 0s 7ms/step - kl_loss: 5.1264e-05
Epoch 10/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 4.5492e-05
Epoch 11/20
50/50 [==============================] - 0s 8ms/step - kl_loss: 3.9672e-05
Epoch 12/20
50/50 [==============================] - 0s 7ms/step - kl_l

In [8]:
res = m((ob_seqs[0:10], None))
res

[<tf.Tensor: shape=(10, 2), dtype=float32, numpy=
 array([[-0.4788724 , -0.00150573],
        [-0.49929467, -0.00390803],
        [-0.5756453 , -0.00317406],
        [-0.47596428, -0.00462553],
        [-0.5494913 , -0.00241544],
        [-0.45684555, -0.00295337],
        [-0.544863  ,  0.00148448],
        [-0.49872845,  0.00403752],
        [-0.5279208 ,  0.00194323],
        [-0.47966886, -0.00317384]], dtype=float32)>,
 <tf.Tensor: shape=(10, 2), dtype=float32, numpy=
 array([[0.999948  , 0.9997817 ],
        [1.0007331 , 1.0010453 ],
        [0.9993722 , 1.0002307 ],
        [1.0019991 , 0.99951524],
        [0.9989625 , 0.9992386 ],
        [1.0023005 , 0.99829125],
        [0.99854785, 1.0023077 ],
        [1.0012319 , 1.0012643 ],
        [1.0011668 , 1.0026312 ],
        [1.0011507 , 0.99877083]], dtype=float32)>,
 <tf.Tensor: shape=(10, 30), dtype=float32, numpy=
 array([[-0.20079066,  0.38631848, -0.0954698 , -0.1312818 , -0.15134756,
         -0.01042288, -0.18632105,  0.0

In [9]:
next_obs[0:10]

array([[-0.47832373, -0.00274939],
       [-0.49251005, -0.005223  ],
       [-0.57791984, -0.00141938],
       [-0.47088268, -0.00629681],
       [-0.55019915, -0.00119392],
       [-0.45699415, -0.00656845],
       [-0.54774845,  0.00525958],
       [-0.4956426 ,  0.00690681],
       [-0.52634752,  0.00604456],
       [-0.47780034, -0.00620174]])